KEYS: 相较于实盘，简化了很多逻辑，给出一个版本

1. 假设网格间距不变
2. 价格达到预设价格后才发单，则没有撤单逻辑

In [1]:
import dai
import numpy as np
import pandas as pd
from datetime import datetime
from bigmodule import M
from bigtrader.constant import OrderStatus, Direction, Offset

def initialize(context):
    # 策略参数
    context.short_grids = (0, 0)
    context.long_grids = (6000, 6500)
    context.grid_interval = 10
    context.order_qty = 5
    context.cancel_parameter = 1
    context.close_interval = context.grid_interval

    # 策略变量
    context.symbol = 'SR409.CZC'    # 交易合约
    context.curr_grid = 0           # 当前网格线
    context.next_grid = 0           # 下个网格线
    context.grid_info = {}          # 已开仓网格信息
    context.order_flag = 0          # 交易方向：1-做多；-1-做空
    context.stats_info_sum = {}     # 每日统计（全部）
    context.stats_info = []         # 每日统计：开仓次数和平仓次数
    context.isprint = 1             # 日志打印

def before_trading_start(context, data):
    context.stats_info = [0, 0]     # 重置每日统计结果
    context.subscribe(context.symbol)

    # 每日盘前初始化策略变量
    if not context.grid_info:
        # 第一次则判断昨日收盘价
        last_close = data.history(context.symbol, ['close'], 1, '1d')['close'].values[0]
        if (last_close >= context.short_grids[0]) and (last_close <= context.short_grids[1]):
            context.curr_grid = context.short_grids[0]
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif (last_close >= context.long_grids[0]) and (last_close <= context.long_grids[1]):
            context.curr_grid = context.long_grids[1]
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    else:
        min_grid = min(context.grid_info)
        max_grid = max(context.grid_info)
        if (min_grid >= context.short_grids[0]) and (max_grid <= context.short_grids[1]):
            context.curr_grid = max_grid
            context.next_grid = context.curr_grid + context.grid_interval
            context.order_flag = -1
        elif (min_grid >= context.long_grids[0]) and (max_grid <= context.long_grids[1]):
            context.curr_grid = min_grid
            context.next_grid = context.curr_grid - context.grid_interval
            context.order_flag = 1
    
    msg = '【盘前初始化】{} 网格方向: {}, 当前网格: {}, 下个网格: {}'.format(data.current_dt, context.order_flag, context.curr_grid, context.next_grid)
    context.write_log(msg, stdout=context.isprint)

def send_order(context, direction: int, offset: int, price: float, volume: int):
    if (direction == Direction.LONG) and (offset == Offset.OPEN):          # 买开
        if volume != 0:
            context.buy_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.OPEN):       # 卖开
        if volume != 0:
            context.sell_open(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[0] += 1
    elif (direction == Direction.LONG) and (offset == Offset.CLOSE):        # 买平
        if volume != 0:
            context.buy_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid + context.grid_interval
        context.stats_info[1] += 1
    elif (direction == Direction.SHORT) and (offset == Offset.CLOSE):       # 卖平
        if volume != 0:
            context.sell_close(context.symbol, volume, price)
        context.curr_grid = price
        context.next_grid = context.curr_grid - context.grid_interval
        context.stats_info[1] += 1
    msg = '【发单】方向：{}, 开平: {}, 价格: {}, 数量: {}, 当前网格: {}, 下个网格: {}'.format(direction, offset, price, volume, context.curr_grid, context.next_grid)
    context.write_log(msg, stdout=context.isprint)

    # 更新已开仓网格信息
    if offset == 1:
        context.grid_info[price] = 0

def handle_tick(context, tick):
    if ((tick.time_int > 205000000) & (tick.time_int < 205999999)) or ((tick.time_int > 85500000) & (tick.time_int < 85999999)):
        return
    # 做多逻辑
    if context.order_flag == 1:
        # print(f"test: {tick.datetime}, {tick.ask_price1}, {tick.bid_price1}, {context.curr_grid}, {context.next_grid}")
        if tick.bid_price1 < context.next_grid:        # 做多开仓 - 买开
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发买开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.LONG, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.ask_price1 > context.curr_grid + context.close_interval:       # 做多平仓 - 卖平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发卖平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.SHORT, offset=Offset.CLOSE,
                price=context.curr_grid + context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )
    elif context.order_flag == -1:
        if tick.ask_price1 >= context.next_grid:       # 做空开仓 - 卖开
            if context.next_grid in context.grid_info:
                return
            msg = f'【触发卖开】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(context, direction=Direction.SHORT, offset=Offset.OPEN, price=context.next_grid, volume=context.order_qty)
        elif tick.bid_price1 < context.curr_grid - context.close_interval:       # 做空平仓 - 买平
            if context.curr_grid not in context.grid_info:
                return
            msg = f'【触发买平】{tick.datetime}。卖一价: {tick.ask_price1}, 买一价: {tick.bid_price1}, 当前网格: {context.curr_grid}, 下个网格: {context.next_grid}'
            context.write_log(msg, stdout=context.isprint)
            send_order(
                context, direction=Direction.LONG, offset=Offset.CLOSE,
                price=context.curr_grid - context.close_interval,
                volume=context.grid_info[context.curr_grid]
            )

def handle_order(context, order):
    context.write_log('【委托回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 下单价格: {}, 下单数量: {}, 订单状态: {}, 成交数量: {}'.format(
        order.insert_date, order.order_time, order.symbol, order.order_id, order.direction, order.offset, order.order_price,
        order.order_qty, order.order_status, order.filled_qty
    ), stdout=context.isprint)

    # 撤单
    if order.order_status == OrderStatus.CANCELLED:
        if (order.offset == Offset.OPEN) and (context.grid_info[order.order_price] == 0):
            # 开仓网格还没成交则直接从已开仓网格信息中删除
            del context.grid_info[order.order_price]
        # 其他情况：开仓网格部分成交撤单 & 平仓撤单，则没有任何操作

    # 更新已开仓网格信息
    if order.offset == Offset.OPEN:
        context.grid_info[order.order_price] = order.filled_qty
    elif (order.offset == Offset.CLOSE) or (order.offset == Offset.CLOSETODAY):
        grid_price = order.order_price + context.close_interval if order.direction==Direction.LONG else order.order_price - context.close_interval
        if context.grid_info[grid_price] == order.filled_qty:
            del context.grid_info[grid_price]

def handle_trade(context, trade):
    context.write_log('【成交回报】交易时间: {} {}, 标的: {}, 订单id: {}, 方向:{}, 开平: {}, 成交价格: {}, 成交数量: {}'.format(
        trade.trade_date, trade.trade_time, trade.symbol, trade.trade_id, trade.direction, trade.offset, trade.filled_price, trade.filled_qty
    ), stdout=context.isprint)

def after_trading(context, data):
    context.stats_info_sum[data.current_dt.strftime('%Y-%m-%d')] = context.stats_info
    context.write_log(f'【盘后统计】当日已开仓网格: {context.grid_info}', stdout=context.isprint)
    context.write_log(f'【盘后统计】当日成交次数: {context.stats_info_sum}', stdout=context.isprint)

data = {
    'instruments': ['SR409.CZC'],
    'start_date': '2024-06-03 21:00:00',
    'end_date': '2024-06-19 15:00:00',
}

M.bigtrader.v20(
    data=data,
    start_date='',
    end_date='',
    initialize=initialize,
    before_trading_start=before_trading_start,
    handle_tick=handle_tick,
    handle_trade=handle_trade,
    handle_order=handle_order,
    after_trading=after_trading,
    capital_base=10000001,
    frequency='tick',
    product_type='期货',
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy='open',
    order_price_field_sell='open',
    benchmark='000300.SH',
    plot_charts=True,
    debug=True,
    backtest_only=False,
    m_cached=False
)

[2024-06-20 09:41:31] [info     ] bigtrader.v20 开始运行 ..
[2024-06-20 09:41:31] [info     ] 2024-06-03 21:00:00, 2024-06-19 15:00:00, instruments=1
[2024-06-20 09:41:32] [info     ] bigtrader module V2.0.3
[2024-06-20 09:41:32] [info     ] bigtrader engine v1.10.9 2024-06-17
[2024-06-20 09:41:32] [info     ] begin reading history data, 2024-06-03 21:00:00~2024-06-19 15:00:00, disable_cache:1
[2024-06-20 09:41:32] [info     ] reading benchmark data 000300.SH 2024-06-03 21:00:00~2024-06-19 15:00:00...
[2024-06-20 09:41:32] [info     ] reading daily data 2024-05-31 00:00:00~2024-06-19 15:00:00...
[2024-06-20 09:41:32] [info     ] cached_benchmark_ds:dai.DataSource("_858a61ccaaed488481551f55ddac6225")
[2024-06-20 09:41:32] [info     ] cached_daily_ds:None
2024-06-20 09:41:32.901222 init history datas... 
2024-06-20 09:41:32.905058 init history datas done. 
2024-06-20 09:41:32.915092 run_backtest() capital_base:10000001, frequency:tick, product_type:future, date:2024-06-03 21:00:00 ~ 2024-06-

[2024-06-20 09:42:56] [info     ] bigtrader.v20 运行完成 [85.546s].


{'raw_perf': dai.DataSource("_afa397fcb3d64e74b394c25f899a86c2"), 'order_price_field_buy': 'open', 'order_price_field_sell': 'open', 'plot_charts': True, 'start_date': '2024-06-03 21:00:00', 'end_date': '2024-06-19 15:00:00', 'capital_base': 10000001, 'instruments': None, 'frequency': 'tick', 'benchmark': '000300.SH', 'product_type': 'future', 'before_start_days': 0, 'volume_limit': 1, 'daily_data': None, 'benchmark_data': dai.DataSource("_858a61ccaaed488481551f55ddac6225"), 'basic_data': dai.DataSource("_18ea5a63105e404ba34337d14b49345b"), 'dominant_data': None, 'options_data': None, 'debug': True, 'strategy_name': None, 'run_begin_time': '2024-06-20 09:41:31', 'run_mode': 'backtest', 'run_engine': 'py', 'render': <bound method render of {...}>, 'display': <bound method render of {...}>, 'read_raw_perf': <bound method read_raw_perf of {...}>, 'analyze_pnl_per_trade': <bound method analyze_pnl_per_trade of {...}>, 'analyze_pnl_per_day': <bound method analyze_pnl_per_day of {...}>, 'plo